<a href="https://colab.research.google.com/github/guardyourheart/gladiator/blob/main/gladiator_auto111_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚔️ Gladiator WebUI

In [1]:
#@markdown #STEP 1️⃣: Download Gladiator WebUI and req
from IPython.display import clear_output, display, HTML
import time
import os
import subprocess
import shutil
import requests
import threading
import urllib.request
from urllib.parse import unquote
from datetime import timedelta
from google.colab import drive
from IPython.utils import capture
from subprocess import getoutput
from google.colab.output import eval_js

def check_gpu():
    print("\033[96m")  # Cyan text
    print('⌚ Checking GPU...', end='')
    output = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv')
    if "name" in output:
        gpu_name = output[5:]
        print('\r✅ Current GPU:', gpu_name, flush=True)
    else:
        print('\r\033[91m❎ ERROR: No GPU detected. Lakukan Step dibawah ini.\n', flush=True)
        # Additional code for displaying an image and message
        display(HTML("<img src='https://i.ibb.co/HC9KH17/NVIDIA-Share-23-01-02-173037.png' width='800px'/>"))
        print('\033[91m\nJika tertulis "Cannot connect to GPU backend", berarti telah mencapai batas pemakaian. Silahakn istirahat untuk hari ini...')
        display(HTML("<center><img src='https://media.tenor.com/RY9NX67klacAAAAi/sad-cute.gif' width='272px'/></center>"))
        time.sleep(5)
        from google.colab import runtime
        runtime.unassign()

def clone_repositories():
    SD_Next_Commit_Hash = "last commit"  # @param {'type': 'string'}
    #@markdown Leave this if you want lastest version of Stable Diffusion.
    if SD_Next_Commit_Hash == "last commit":
        subprocess.run(["git", "clone", "--depth=1", "https://github.com/guardyourheart/gladiator_webui.git"])
    else:
        subprocess.run(["git", "clone", "https://github.com/guardyourheart/gladiator_webui.git"])
        subprocess.run(["git", "checkout", SD_Next_Commit_Hash], cwd="gladiator_webui")

    !git clone https://github.com/guardyourheart/gladiator_webui.git
    clear_output()

def install_requirements():
    %cd /content/gladiator_webui
    !pip install --use-feature=fast-deps -r requirements.txt
    !pip install timm==0.6.13 --upgrade
    !pip install onnxruntime==1.15.1
    !pip install --upgrade huggingface_hub
    !pip install wget
    !pip install tqdm
    !apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
    clear_output()

def clone_MiDaS():
    %cd /content/gladiator_webui/repositories
    !git clone https://github.com/isl-org/MiDaS MiDaS
    clear_output()

def install_jemalloc():
    %cd /content/gladiator_webui/
    !apt -y update -qq
    !apt -y install libjemalloc-dev
    %env LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so.2
    clear_output()

def connect_to_drive():
    %cd /content
    drive.mount('/content/drive')
    clear_output()

# Check GPU
start_colab = int(time.time()) - 5
check_gpu()

connect_to_drive()

# Clone repositories
clone_repositories()

# Pip Install Req
install_requirements()

# Install jemalloc and its development package
install_jemalloc()

%cd /content/gladiator_webui
!git show --oneline -s
print("✅ Done", "success")

/content/gladiator_webui
8c6f8b7 (grafted, HEAD -> master, origin/master, origin/HEAD) Delete gladiator_auto111_colab.ipynb
✅ Done success


# 🙈 Download Models, Loras etc

In [5]:
def download_file(url, output_path):
    response = subprocess.run(
        ["aria2c", "-x4", "-s4", "-d", output_path, url],
        capture_output=True,
        text=True,
    )

    if response.returncode == 0:
        print("File downloaded successfully:", url)
    else:
        print("Failed to download the file:", url)
        print("Aria2c Error:", response.stderr)

def download_model(Type, DirectLink_URL):
    if Type == "None" or not DirectLink_URL:
        print("Skipping download as no valid Type or DirectLink_URL provided.")
        return

    output_path = "/content/gladiator_webui/models/"

    if Type == "Checkpoint":
        output_path += "Stable-diffusion/"
    elif Type == "LoRA":
        output_path += "LyCORIS/"
    elif Type == "LoCon/LyCORIS":
        output_path += "LyCORIS/"
    elif Type == "Textual Inversion":
        output_path += "embeddings/"
    else:
        print("Invalid type specified.")
        return

    # Create the directories if they don't exist
    os.makedirs(output_path, exist_ok=True)

    urls = [url.strip() for url in DirectLink_URL.split(",")]

    # Use a separate thread for downloading files in the background
    def download_thread():
        threads = []
        for url in urls:
            thread = threading.Thread(target=download_file, args=(url, output_path))
            thread.start()
            threads.append(thread)

        # Wait for all threads to finish
        for thread in threads:
            thread.join()

        # Clear the output when downloads are done
        clear_output()
        print("✅ All downloads completed.")

    download_thread()

# Call the function with the first set of parameters
Type1 = "Checkpoint"  # @param ["None", "Checkpoint", "LoRA", "LoCon/LyCORIS", "Textual Inversion"]
DirectLink_URL1 = "https://civitai.com/api/download/models/152076, https://civitai.com/api/download/models/154593"  # @param {'type': 'string'}
download_model(Type1, DirectLink_URL1)

# Call the function with the second set of parameters
Type2 = "None"  # @param ["None", "Checkpoint", "LoRA", "LoCon/LyCORIS", "Textual Inversion"]
DirectLink_URL2 = ""  # @param {'type': 'string'}
download_model(Type2, DirectLink_URL2)

✅ All downloads completed.
Skipping download as no valid Type or DirectLink_URL provided.


# 👻Download Extensions, ControlNet, VEA, Upscalers


In [6]:
#@markdown #🎲 Pre-Install Extensions

!git clone https://github.com/camenduru/sd-civitai-browser /content/gladiator_webui/extensions/sd-civitai-browser
!git clone https://github.com/guardyourheart/gladiator-webui-controlnet /content/gladiator_webui/extensions/gladiator-webui-controlnet
!git clone https://github.com/guardyourheart/sdxl_aspect_ration /content/gladiator_webui/extensions/sdxl_aspect_ration
!git clone https://github.com/nolanaatama/microsoftexcel-tunnels /content/gladiator_webui/extensions/microsoftexcel-tunnels
!git clone https://github.com/Bing-su/adetailer /content/gladiator_webui/extensions/adetailer
!git clone https://huggingface.co/Bingsu/adetailer /content/gladiator_webui/models/adetailer
!git clone https://github.com/fkunn1326/openpose-editor /content/gladiator_webui/extensions/openpose-editor
!git clone https://github.com/etherealxx/batchlinks-webui /content/gladiator_webui/extensions/batchlinks-webui
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 /content/gladiator_webui/extensions/ultimate-upscale-for-automatic1111
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 /content/gladiator_webui/extensions/multidiffusion-upscaler-for-automatic1111

clear_output()
print("✅ Done", "success")


✅ Done success


In [12]:
#@markdown #🎲 Controlnet for SDXL

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/OpenPoseXL2.safetensors -d /content/gladiator_webui/models/controlnet -o OpenPoseXL2.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank128.safetensors -d /content/gladiator_webui/models/controlnet -o control-lora-canny-rank128.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank256.safetensors -d /content/gladiator_webui/models/controlnet -o control-lora-canny-rank256.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank128.safetensors -d /content/gladiator_webui/models/controlnet -o control-lora-depth-rank128.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank256.safetensors -d /content/gladiator_webui/models/controlnet -o control-lora-depth-rank256.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank128.safetensors -d /content/gladiator_webui/models/controlnet -o control-lora-recolor-rank128.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank256.safetensors -d /content/gladiator_webui/models/controlnet -o control-lora-recolor-rank256.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank128-metadata.safetensors -d /content/gladiator_webui/models/controlnet -o control-lora-sketch-rank128-metadata.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank256.safetensors -d /content/gladiator_webui/models/controlnet -o control-lora-sketch-rank256.safetensors

clear_output()
print("✅ Done", "success")


✅ Done success


In [8]:
#@markdown 🎲 Download VAE Collection (+Negtive Embedings)
%cd /content/gladiator_webui/models
!mkdir -p embeddings/negative
%cd /content/gladiator_webui/models/embeddings/negative/
!git clone --quiet https://huggingface.co/embed/negative

import os
import urllib.request
from IPython.display import clear_output

VAE = "sdxl_vae.safetensors" # @param ["None", "WD-v2.vae.pt", "blessed-fix.vae.pt", "blessed.vae.pt", "blessed2.vae.pt", "clearvae_main.safetensors", "cute_vae.safetensors", "grapefruitVAE_v1.pt", "kl-f8-anime.ckpt", "kl-f8-anime2.ckpt", "nai.vae.pt", "orangemix.vae.pt", "pastel-waifu-diffusion.vae.pt", "rmada-cold-vae.ckpt", "vae-ft-mse-840000-ema-pruned.ckpt", "vaeextremecolors_v10.pt", "sdxl_vae.safetensors"]

VAE_urls = {
    "WD-v2.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/WD-v2.vae.pt',
    "blessed-fix.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/blessed-fix.vae.pt',
    "blessed.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/blessed.vae.pt',
    "blessed2.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/blessed2.vae.pt',
    "clearvae_main.safetensors": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/clearvae_main.safetensors',
    "cute_vae.safetensors": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/cute%20vae.safetensors',
    "grapefruitVAE_v1.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/grapefruitVAE_v1.pt',
    "kl-f8-anime.ckpt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/kl-f8-anime.ckpt',
    "kl-f8-anime2.ckpt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/kl-f8-anime2.ckpt',
    "nai.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/nai.vae.pt',
    "orangemix.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/orangemix.vae.pt',
    "pastel-waifu-diffusion.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/pastel-waifu-diffusion.vae.pt',
    "rmada-cold-vae.ckpt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/rmada-cold-vae.ckpt',
    "vae-ft-mse-840000-ema-pruned.ckpt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt',
    "vaeextremecolors_v10.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/vaeextremecolors_v10.pt',
    "sdxl_vae.safetensors": 'https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors'
}

output_path = "/content/gladiator_webui/models/VAE/"

os.makedirs(output_path, exist_ok=True)  # Create the output directory if it doesn't exist

def download_model(model_url, output_path):
    filename = os.path.basename(model_url)
    output_file = os.path.join(output_path, filename)
    urllib.request.urlretrieve(model_url, output_file)
    return filename

model_url = VAE_urls.get(VAE)
if model_url:
    download_model(model_url, output_path)
    print("✓ Done", "success")
else:
    clear_output()
    print("Selected VAE model not found in the list.")

# Clear the output to keep the notebook clean
clear_output()
print("✅ Done", "success")

✅ Done success


In [9]:
#@markdown #🎲Download UPSCALERs

# Set the name of the upscaler you want to download here
selected_upscaler = "8x_NMKD-Superscale_150000_G.pth" # @param ["None", "4x_NMKD-Superscale-SP_178000_G.pth", "4x-UltraSharp.pth", "4x_CountryRoads_377000_G.pth", "4x_Fatality_Comix_260000_G.pth", "4x_RealisticRescaler_100000_G.pth", "4x_Valar_v1.pth", "4x_fatal_Anime_500000_G.pth", "4x_foolhardy_Remacri.pth", "A_ESRGAN_Single.pth", "LADDIER1_282500_G.pth", "WaifuGAN_v3_30000.pth", "lollypop.pth", "sudo_rife4_269.662_testV1_scale1.pth", "8x_NMKD-Superscale_150000_G.pth"]


upscalers_urls = {
    "4x_NMKD-Superscale-SP_178000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NMKD-Superscale-SP_178000_G.pth',
    "4x-UltraSharp.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x-UltraSharp.pth',
    "4x_CountryRoads_377000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_CountryRoads_377000_G.pth',
    "4x_Fatality_Comix_260000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_Fatality_Comix_260000_G.pth',
    "4x_RealisticRescaler_100000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_RealisticRescaler_100000_G.pth',
    "4x_Valar_v1.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_Valar_v1.pth',
    "4x_fatal_Anime_500000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_fatal_Anime_500000_G.pth',
    "4x_foolhardy_Remacri.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_foolhardy_Remacri.pth',
    "A_ESRGAN_Single.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/A_ESRGAN_Single.pth',
    "LADDIER1_282500_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/LADDIER1_282500_G.pth',
    "WaifuGAN_v3_30000.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/WaifuGAN_v3_30000.pth',
    "lollypop.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/lollypop.pth',
    "sudo_rife4_269.662_testV1_scale1.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/sudo_rife4_269.662_testV1_scale1.pth',
    "8x_NMKD-Superscale_150000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/8x_NMKD-Superscale_150000_G.pth',
}

output_path = "/content/gladiator_webui/models/ESRGAN/"

os.makedirs(output_path, exist_ok=True)

def download_model(model_url, output_path):
    if model_url is None:
        return None

    filename = os.path.basename(model_url)
    output_file = os.path.join(output_path, filename)
    urllib.request.urlretrieve(model_url, output_file)
    return filename

model_url = upscalers_urls.get(selected_upscaler)

if model_url is not None:
    download_model(model_url, output_path)
    print("✓ Done - Success")
else:
    clear_output()
    print("Invalid upscaler selected. Finish...")

clear_output()
print("✅ Done - Success")

✅ Done - Success


---

# ⚡Start Gladiator WebUI

In [ ]:
# @title ⚡Start Gladiator WebUI

# --Function AntiDissconnect
%cd /content/gladiator_webui
clear_output()
audio_url = "https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a"
# Function AntiDissconnect in the background
def play_audio(url):
    display(HTML(f'<audio src="{url}" controls autoplay style="display:none"></audio>'))
# Create a separate thread for AntiDissconnect
audio_thread = threading.Thread(target=play_audio, args=(audio_url,))
audio_thread.start()

%env LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so.2
%env TF_CPP_MIN_LOG_LEVEL=1

!python launch.py --no-half-vae --xformers --medvram-sdxl --enable-insecure-extension-access --ngrok 2NvXfMWtoWSR4BSK0kPINmej7yz_657hrUZX4vbQwVcY2EAKE

env: LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so.2
env: TF_CPP_MIN_LOG_LEVEL=1


fatal: No names found, cannot describe anything.
Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Version: 1.6.0
Commit hash: 8c6f8b77888067b8dd3e6662730af1d1cd8990c4
Launching Web UI with arguments: --no-half-vae --xformers --medvram-sdxl --enable-insecure-extension-access --ngrok 2NvXfMWtoWSR4BSK0kPINmej7yz_657hrUZX4vbQwVcY2EAKE
2023-09-05 17:47:48.487304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ngrok authtoken detected, trying to connect...
ngrok connected to localhost:7860! URL: https://2124-34-126-141-219.ngrok-free.app
You can use this link after the launch is complete.
[-] ADetailer initialized. version: 23.9.1, num models: 15
2023-09-05 17:48:01,443 - ControlNet - INFO - ControlNet v1.1.313
ControlNet preprocessor location: /content/gladiator_webui/extensions/gladiator-webui-controlnet/annotator/downloads
2023-09-05 17:48:01,658 - ControlNet - INFO - ControlNet v1.1.313
Loading weights [700528894b] from /content/g